# Object Reorientation with Help of Environment Contact

This example demonstrates a Franka Emika Panda robot sequentially manipulating multiple objects. The robot follows a structured plan:

1. Start from a ready position
2. Move above an object from the list
3. Grasp the object
4. Execute a planned movement with the object
5. Release the object
6. Move 10 cm above the current position
7. Return to ready position
8. Repeat for the next object in the list until all objects are manipulated

In [ ]:
import genesis as gs
import numpy as np
import os
import torch
from pytransform3d import (
    transformations as pt,
    rotations as pr,
    batch_rotations as pb,
    trajectories as ptr,
    plot_utils as ppu
)
from pandaSim.geometry.genesis_adapter import GenesisAdapter
from pandaSim.planning.screw_motion_planner import ScrewMotionPlanner
from utils import RobotController

import spatialmath as sm
# auto reload
%load_ext autoreload
%autoreload 2

## Initialize Robot Controller

We'll create a RobotController instance that will handle all the robot operations and scene management.

`Note:`  `compile_kernels` set whether to compile the simulation kernels inside `build()`. If False, the kernels will not be compiled (or loaded if found in the cache) until the first call of `scene.step()`. This is useful for cases you don't want to run the actual simulation, but rather just want to visualize the created scene. Kernel Compilation may take anytime between 2-10 minutes depending on your system.

In [ ]:


# Record the video
record = True
video_path = "video/"
os.makedirs(video_path, exist_ok=True)

# Initialize the scene and get the robot and objects
# Initialize the robot controller
controller = RobotController()
franka, objects_list, target_entity, scene_list, cam = controller.init_scene(record=record)
scene, adapter = controller.scene, controller.adapter
# Get the finger link for manipulation
finger_link = franka.get_link('virtual_finger')

# Build the scene
scene.build(compile_kernels=False)

## Define Ready Pose

Set up the robot's ready pose configuration.

In [16]:
def reset_scene():
    scene.reset()

    # Define ready pose joint positions
    qr = np.array([0.0000, -0.3002, 0.0000, -2.1991, 0.0000, 2.0071, 0.7854, 0.04, 0.04])
    ready_qpos = np.append(qr, [0, 0, 0])  # Adding positions for virtual finger and spinning pads

    # Set robot to ready pose
    franka.ready_qpos = ready_qpos.squeeze()
    franka.set_qpos(franka.ready_qpos)
    franka.control_dofs_position(franka.ready_qpos)
    
    # Visualize initial position
    target_entity.set_pos(finger_link.get_pos())
    target_entity.set_quat(finger_link.get_quat())
    
    scene.step()

## Main Execution Loop

Now we'll process each object in the list sequentially, following our plan:
1. Move to above the object
2. Grasp the object
3. Execute the manipulation plan
4. Release the object
5. Move 10cm up
6. Return to ready pose
7. Process next object
- Note: Running the following cell can take about 2-10 minutes.

In [ ]:
objects_to_manipulate = objects_list.copy()
initial_pose = []
goal_pose = []
current_pose = []
# Reset the scene to initial state
reset_scene()
record = True

if cam and record:
    cam.start_recording()
    print("Recording started")


# Process each object in the list
while objects_to_manipulate:
    # Get the next object from the list
    obj = objects_to_manipulate.pop(0)
    obj_name = obj.name

    
    print(f"\nProcessing {obj_name}...")
    
    # Step 1: Compute grasp pose for the object
    print(f"Computing grasp pose for {obj_name}")
    grasp_height = 'center' if obj_name == 'cylinder' else 'top'
    offset_toward = 0.04 if obj_name == 'bottle' else 0.02
    offset_upward = 0.01

    prefer_closer_grasp = False if obj_name == 'cube' else True
    grasp_pose, qs, s_axes = controller.compute_object_grasp(
        obj=obj,
        grasp_height=grasp_height,
        offset_toward=offset_toward,
        offset_upward=offset_upward,
        prefer_closer_grasp=prefer_closer_grasp
    )
    
    # Step 2: Grasp the object
    print(f"Moving to grasp {obj_name}")
    fingers_state = 0.02 if obj_name == 'bottle' else 0.04
    controller.pregrasp_object(finger_link, grasp_pose, fingers_state=fingers_state)
    object_trajectory = controller.object_trajectory(finger_link, obj, qs, s_axes)
    controller.grasp_object(finger_link, grasp_pose, fingers_state=fingers_state, fingers_force=2)
    
    # Step 3: Execute manipulation plan
    print(f"Executing manipulation with {obj_name}")
    controller.execute_manipulation(finger_link, obj, qs, s_axes)
    
    # Step 4: Release the object
    print(f"Releasing {obj_name}")
    controller.release_object()
    
    # Step 5: Move 10cm up from the current position
    print(f"Moving up from {obj_name}")
    controller.move_up_from_current(finger_link, 0.1)
    
    
    # Step 6: Return to ready pose
    # print(f"Returning to ready pose")
    # controller.return_to_ready_pose()
    
    print(f"Completed processing {obj_name}")


    # save poses
    current_pose.append(controller.adapter.get_pose(obj, 't'))
    initial_pose.append(object_trajectory[0])
    goal_pose.append(object_trajectory[-1])

if cam and record:
    cam.stop_recording(save_to_filename=os.path.join(video_path, "upright.mp4"))
    
print("\nAll objects manipulated successfully!")

In [ ]:
es = []
Successes = []
arriveds = []
for i, obj in enumerate(objects_list):
    e, Success, arrived = controller.Success_Rate(initial_pose[i], goal_pose[i], current_pose[i], 'axis')
    es.append(e)
    Successes.append(Success)
    arriveds.append(arrived)
    print(f"Object {obj.name} Success Rate: {Success:.2f}%")


    # save all info in log file, append to the file
    with open('log.txt', 'a') as f:
        f.write(f"\nObject {obj.name} Success Rate: {Success:.2f}%\n")
        f.write(f"Object {obj.name} Arrived: {arrived}\n")
        f.write(f"Object {obj.name} Error: {e}\n")
        f.write(f"Initial Pose:\n {initial_pose[i]}\n")
        f.write(f"Goal Pose:\n {goal_pose[i]}\n")
        f.write(f"Current Pose:\n {current_pose[i]}\n")
        f.write("-"*100)
        f.write("\n")

with open('log.txt', 'a') as f:
    f.write("="*100)
    f.write("\n")



